In [8]:
from utils import *

import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
x = torch.randn(32,3,224,224) # b c h w
show(x)

x.shape = torch.Size([32, 3, 224, 224])


In [ ]:
PATCH_SIZE = 16

def patch_embedding(x):
    b, c, h, w = x.shape
    assert h % PATCH_SIZE == 0
    assert w % PATCH_SIZE == 0

    n_h = h // PATCH_SIZE
    n_w = w // PATCH_SIZE

    embed_dim = c * PATCH_SIZE**2

    conv_proj = nn.Conv2d(
        in_channels=c,
        out_channels=embed_dim,
        kernel_size=PATCH_SIZE,
        stride=PATCH_SIZE,
    )

    show(x, "input")

    x = conv_proj(x)     # b n_x n_h n_w
    show(x)

    x = x.flatten(2)     # b n_x n_t
    show(x)

    x = x.permute(0,2,1) # b n_t n_x
    show(x)

    cls_token = nn.Parameter(torch.zeros(1,1,embed_dim))
    cls_token = cls_token.expand(b,-1,-1)

    x = torch.cat([cls_token, x], dim=1) # b (1+n_t) n_x
    show(x, "output")

    return x

x = patch_embedding(x)

x.shape = torch.Size([32, 3, 224, 224]) <- input


x.shape = torch.Size([32, 768, 14, 14])
x.shape = torch.Size([32, 768, 196])
x.shape = torch.Size([32, 196, 768])
x.shape = torch.Size([32, 197, 768]) <- output


In [ ]:
def multi_head_attention(x, kdim, vdim, num_heads):
    b, n_t, n_x = x.shape
    assert n_x % num_heads == 0

    q_proj = nn.Linear(n_x, kdim)
    k_proj = nn.Linear(n_x, kdim)
    v_proj = nn.Linear(n_x, vdim)

def encoder_block(x):
    return x

In [ ]:
tiheadAttention()